# Generate moment coefficients

The following steps are performed to obtain all the artifacts for force coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

In [1]:
from cfdmod.use_cases.pressure.path_manager import CmPathManager
import pathlib

path_manager = CmPathManager(output_path=pathlib.Path("./output/pressure"))

Read parameters file

In [2]:
from cfdmod.use_cases.pressure.moment.Cm_config import CmConfig

config_path=pathlib.Path("./fixtures/tests/pressure/Cm_params.yaml")
post_proc_cfg = CmConfig.from_file(config_path)

post_proc_cfg

{'measurement_1': CmConfig(bodies={'marquise': BodyConfig(surfaces=['list_of_surfaces_of_marquise'], sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf])), 'lanternim': BodyConfig(surfaces=['list_of_surfaces_of_lanternim'], sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf]))}, variables=['Cmx', 'Cmy', 'Cmz'], statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']),
 'measurement_2': CmConfig(bodies={'building': BodyConfig(surfaces=['building'], sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf]))}, variables=['Cmx', 'Cmy', 'Cmz'], statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'])}

Read mesh from file

In [3]:
from nassu.lnas import LagrangianFormat

mesh_path=pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LagrangianFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read pressure coefficient data

In [ ]:
import pandas as pd

cp_data_path=pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")
cp_data = pd.read_hdf(cp_data_path)

cp_data_to_use = (
    cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data
)
cp_data_to_use

Add areas to pressure coefficient data

In [ ]:
vec_areas = mesh.geometry._cross_prod() / 2
areas_df = pd.DataFrame({"Ax": vec_areas[:, 0], "Ay": vec_areas[:, 1], "Az": vec_areas[:, 2]})
areas_df["point_idx"] = areas_df.index

cp_data = pd.merge(cp_data, areas_df,
    on="point_idx",
    how="left"
)

Calculate moment coefficient

In [ ]:
from cfdmod.use_cases.pressure.force.Cf_data import process_body

for cfg_label, cfg in post_proc_cfg.items():
    for body_label, body_cfg in cfg.bodies.items():
        processed_body = process_body(mesh=mesh, body_cfg=body_cfg, cp_data=cp_data, cfg=cfg)
        processed_body.save_outputs(body_label=body_label, cfg_label=cfg_label, path_manager=path_manager)